In [2]:
import os
import os.path
# import torch as torch #I AM USING PYTHORCH; SIMILAR WITH TENSORFLOW, JUST USUAL IMPORTS
# import torch.nn as nn #I AM USING PYTHORCH; SIMILAR WITH TENSORFLOW, JUST USUAL IMPORTS
# import torch.nn.functional as F #I AM USING PYTHORCH; SIMILAR WITH TENSORFLOW, JUST USUAL IMPORTS
# import torch.optim as optim #I AM USING PYTHORCH; SIMILAR WITH TENSORFLOW, JUST USUAL IMPORTS
import tensorflow as tf
import numpy as np
from art.utils import load_dataset
from art.attacks import *
from art.utils import load_dataset
from art.classifiers import PyTorchClassifier
from art.utils import load_mnist
from art.classifiers import PyTorchClassifier
from art.utils import load_mnist
from art.utils import load_cifar10
# import torchvision
# import torchvision.transforms as transforms
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from matplotlib.pyplot import imshow

parametri di configurazione

In [ ]:
IMAGE=10 #DATO CHE HO UN ARRAY DI 10000 IMMAGINI, QUESTA E' L'IMMAGINE CONSIDERATA

In [ ]:
#mnist #I AM USING THE FOLLOWING MODEL
class Net(nn.Module):
	def __init__(self):
		super(Net, self).__init__()
		self.conv_1 = nn.Conv2d(in_channels=1, out_channels=4, kernel_size=5, stride=1)
		self.conv_2 = nn.Conv2d(in_channels=4, out_channels=10, kernel_size=5, stride=1)
		self.fc_1 = nn.Linear(in_features=4 * 4 * 10, out_features=100)
		self.fc_2 = nn.Linear(in_features=100, out_features=10)

	def forward(self, x):
		x = F.relu(self.conv_1(x))
		x = F.max_pool2d(x, 2, 2)
		x = F.relu(self.conv_2(x))
		x = F.max_pool2d(x, 2, 2)
		x = x.view(-1, 4 * 4 * 10)
		x = F.relu(self.fc_1(x))
		x = self.fc_2(x)
		return x

In [ ]:
PATH = 'mnist.pth' #PATH TO THE MODEL, PREVIOUSLY TRAINED AND SAVED
model = Net()
(x_train, y_train), (x_test, y_test), min_pixel_value, max_pixel_value = load_dataset('mnist') #THIS LOADS THE MNIST DATASET.
#THE ONLY IMPORTANT THINGS ARE:
#the target image to be tested must be a numpy array of array, i.e.: i) convert image to numpy; ii) put into an array of 1 cell: [image]
# min_pixel_value can be set to 0
# max_pixel_value should be 1 or 255, depends on the image format

x_test1 = np.swapaxes(x_test, 1, 3).astype(np.float32) #TO BE REMOVED IF AXES ARE NOT SWAPPED DURING TRAINING
criterion = nn.CrossEntropyLoss() #TO BE CONFIGURED
optimizer = optim.Adam(model.parameters(), lr=0.01) #TO BE CONFIGURED
classifier = PyTorchClassifier(
	model=model,
	clip_values=(min_pixel_value, max_pixel_value),
	loss=criterion,
	optimizer=optimizer,
	input_shape=(1, 28,28),
	nb_classes=10,
)
model.load_state_dict(torch.load(PATH)) #LOAD THE MODEL
model.eval() #PUT INTO EVAL FORM

Modelli caricati. Ora posso provare la modalità normale e attacco 

In [ ]:
#immagine normale
imshow(np.asarray(x_test[IMAGE])) #CLEAN IMAGE

In [ ]:
attack = DeepFool(classifier=classifier) #THIS LINE GENERATE THE DEEP FOOL ATTACK. SOME ALTERNATIVES:
#FastGradientMethod CarliniL2Method NewtonFool DeepFool ElasticNet
#ProjectedGradientDescentPyTorch
#SpatialTransformation BasicIterativeMethod 
IMAGE_ADV = attack.generate([x_test1[IMAGE]], [y_test[IMAGE]])
IMAGE_ADV1 = np.swapaxes(IMAGE_ADV, 1, 3).astype(np.float32)
imshow(np.asarray(IMAGE_ADV1[0])) #SHOW ADVERSARIAL IMAGE

In [ ]:
print(y_test[IMAGE]) #ground truth
print(classifier.predict([x_test1[IMAGE]])) # PREDICT ON NORMAL IMAGE
print(classifier.predict(IMAGE_ADV))#PREDICT ON ADVERSARIAL IMAGE